# NeRF Rendering from Checkpoint

**Load your trained NeRF model and render test images**

**Time:** ~5-10 minutes

## Step 1: Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nCUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Upload Your results.zip

Click the folder icon on the left → Upload → Select your `results.zip` file

In [ ]:
# Verify upload
import os

if os.path.exists('results.zip'):
    print("✓ results.zip found!")
    !ls -lh results.zip
else:
    print("✗ results.zip not found. Please upload it first.")

## Step 3: Extract Results

In [ ]:
# Extract the zip file
!unzip -q results.zip

print("✓ Extracted")
print("\nContents:")
!ls -lh results/logs/lego_metrics/ 2>/dev/null || !ls -lh logs/lego_metrics/ 2>/dev/null || echo "Checking structure..."
!ls results/ 2>/dev/null || !ls logs/ 2>/dev/null

## Step 4: Mount Google Drive for Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✓ Drive mounted")

## Step 5: Setup Dataset

In [ ]:
# Copy dataset
!mkdir -p data/nerf_synthetic
!cp -r /content/drive/MyDrive/data/nerf_synthetic/lego data/nerf_synthetic/

print("✓ Dataset copied")
!ls data/nerf_synthetic/lego/

## Step 6: Fix Dataset JSON Files

In [ ]:
import json
import os

# Fix JSON files to match actual image filenames
for split in ['train', 'val', 'test']:
    json_path = f'data/nerf_synthetic/lego/transforms_{split}.json'
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Get actual files
    split_dir = f'data/nerf_synthetic/lego/{split}'
    actual_files = sorted([f.replace('.png', '') for f in os.listdir(split_dir) 
                          if f.endswith('.png') and 'depth' not in f])
    
    # Update frame paths
    for i, frame in enumerate(data['frames']):
        if i < len(actual_files):
            data['frames'][i]['file_path'] = f"./{split}/{actual_files[i]}"
    
    # Save
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"✓ Fixed {split}")

print("\n✓ Dataset ready!")

## Step 7: Clone NeRF Code

In [ ]:
# Clone your NeRF repo
!git clone https://github.com/mabuhann/nerf-pytorch.git nerf_code

print("✓ Code cloned")
!ls nerf_code/

## Step 8: Install Dependencies

In [ ]:
!pip install imageio imageio-ffmpeg configargparse scikit-image -q

print("✓ Dependencies installed")

## Step 9: Find the Checkpoint

In [ ]:
import glob

# Find checkpoint file
# Structure: results/logs/lego_metrics/010000/*.tar
checkpoints = glob.glob('results/logs/lego_metrics/010000/*.tar')

if not checkpoints:
    # Try alternate locations
    checkpoints = glob.glob('logs/lego_metrics/010000/*.tar')

if checkpoints:
    checkpoint = checkpoints[0]
    print(f"✓ Found checkpoint: {checkpoint}")
else:
    print("✗ No checkpoint found!")
    print("\nLet's check what was extracted:")
    !ls -R results/ 2>/dev/null || !ls -R logs/ 2>/dev/null

## Step 10: Render Test Images

**This will render 10 test images using your trained model**

In [ ]:
# Create render script
render_script = f"""
import sys
sys.path.insert(0, 'nerf_code')

import torch
import numpy as np
import imageio
import os
from tqdm import tqdm

# Import NeRF functions
from run_nerf_helpers import *
from load_blender import load_blender_data

# Load dataset
print("Loading dataset...")
images, poses, render_poses, hwf, i_split = load_blender_data(
    'data/nerf_synthetic/lego', 
    half_res=False, 
    testskip=1
)
i_train, i_val, i_test = i_split
H, W, focal = hwf
H, W = int(H), int(W)

# Only render first 10 test images
test_indices = i_test[:10]

print(f"Rendering {{len(test_indices)}} test images...")

# Load checkpoint
print("Loading checkpoint...")
checkpoint = torch.load('{checkpoint}', map_location='cpu')

# Create model
render_kwargs_test = {{}}

# Model architecture from original training
embed_fn, input_ch = get_embedder(10, 0)
embeddirs_fn, input_ch_views = get_embedder(4, 0)

model = NeRF(
    D=8, W=256,
    input_ch=input_ch, 
    output_ch=4, 
    skips=[4],
    input_ch_views=input_ch_views, 
    use_viewdirs=True
).cuda()

model_fine = NeRF(
    D=8, W=256,
    input_ch=input_ch, 
    output_ch=4, 
    skips=[4],
    input_ch_views=input_ch_views, 
    use_viewdirs=True
).cuda()

# Load weights
model.load_state_dict(checkpoint['network_fn_state_dict'])
model_fine.load_state_dict(checkpoint['network_fine_state_dict'])

model.eval()
model_fine.eval()

# Setup rendering kwargs
render_kwargs_test['network_fn'] = model
render_kwargs_test['network_fine'] = model_fine
render_kwargs_test['embed_fn'] = embed_fn
render_kwargs_test['embeddirs_fn'] = embeddirs_fn
render_kwargs_test['N_samples'] = 64
render_kwargs_test['N_importance'] = 128
render_kwargs_test['perturb'] = False
render_kwargs_test['white_bkgd'] = True
render_kwargs_test['raw_noise_std'] = 0.

# Create output directory
os.makedirs('nerf_renders', exist_ok=True)

# Render images
with torch.no_grad():
    for idx in tqdm(test_indices):
        pose = poses[idx, :3, :4]
        
        # Render
        rgb, disp, acc, extras = render(
            H, W, focal, 
            chunk=1024*32,
            c2w=pose,
            **render_kwargs_test
        )
        
        # Save
        rgb8 = (np.clip(rgb.cpu().numpy(), 0, 1) * 255).astype(np.uint8)
        filename = f'nerf_renders/test_{{idx:03d}}.png'
        imageio.imwrite(filename, rgb8)

print(f"\\n✓ Rendered {{len(test_indices)}} images to nerf_renders/")
"""

# Save and run
with open('render_nerf.py', 'w') as f:
    f.write(render_script)

print("✓ Render script created")
print("\nRunning rendering...\n")

!python render_nerf.py

## Step 11: View Rendered Images

In [ ]:
from IPython.display import Image, display
import glob

# Get rendered images
rendered = sorted(glob.glob('nerf_renders/*.png'))

print(f"Rendered {len(rendered)} images:\n")

for img in rendered:
    print(f"\n{img}:")
    display(Image(img, width=400))

## Step 12: Download Images

In [ ]:
from google.colab import files

# Zip the renders
!zip -r nerf_rendered_images.zip nerf_renders/

print("✓ Images zipped")

# Download
files.download('nerf_rendered_images.zip')

print("\n✓ Download complete!")
print("\nYou now have NeRF rendered images for your comparison!")